In [5]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import chrome
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [11]:
options = webdriver.ChromeOptions()
options.add_experimental_option("detach", True)
driver = webdriver.Chrome(options=options , service=Service(ChromeDriverManager().install()))
driver.get('https://www.ad.co.il/nadlanprice')
search = driver.find_element(By.ID, 'tbSearchTerm')
search.send_keys('מודיעין')
search_box = driver.find_element(By.ID, 'btnSearchFilter')
search_box.send_keys(Keys.RETURN)

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=114.0.5735.110)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00DAA813+48355]
	(No symbol) [0x00D3C4B1]
	(No symbol) [0x00C45220]
	(No symbol) [0x00C6D046]
	(No symbol) [0x00C6C8B1]
	(No symbol) [0x00C8A73C]
	(No symbol) [0x00C69A36]
	(No symbol) [0x00C8AA94]
	(No symbol) [0x00C9C922]
	(No symbol) [0x00C8A536]
	(No symbol) [0x00C682DC]
	(No symbol) [0x00C693DD]
	GetHandleVerifier [0x0100AABD+2539405]
	GetHandleVerifier [0x0104A78F+2800735]
	GetHandleVerifier [0x0104456C+2775612]
	GetHandleVerifier [0x00E351E0+616112]
	(No symbol) [0x00D45F8C]
	(No symbol) [0x00D42328]
	(No symbol) [0x00D4240B]
	(No symbol) [0x00D34FF7]
	BaseThreadInitThunk [0x761300C9+25]
	RtlGetAppContainerNamedObjectPath [0x77077B4E+286]
	RtlGetAppContainerNamedObjectPath [0x77077B1E+238]


In [12]:
try:
    viewContainer = WebDriverWait(driver , 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "viewContainer"))
        print(element.text)
    )
except:
    driver.quit()


SyntaxError: invalid syntax (3222754451.py, line 4)